In [1]:
# ============================================================
# HYBRID SARIMA-LSTM: Seasonal Demand Forecasting (Per Category)
# Dataset: consolidated_file_cleaned_v2.csv
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# 1. Load dataset
# -----------------------------
FILE = "consolidated_file_cleaned_v2.csv"
df = pd.read_csv(FILE)

# Clean and prepare
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values(by='time')
df = df[df['suitable_for_seasonal_analysis'] == True]

# Focus on monthly demand column
df['month'] = df['time'].dt.to_period('M').dt.to_timestamp()
category_data = df.groupby(['month','second-level_category'])['sold/m'].sum().reset_index()

# -----------------------------
# 2. SARIMA fitting helper
# -----------------------------
def safe_sarima_fit(ts):
    """Attempt SARIMA fitting with fallback parameters."""
    sarima_orders = [
        ((1,1,1), (1,1,1,12)),
        ((0,1,1), (1,1,1,12)),
        ((1,1,0), (0,1,1,12)),
        ((0,1,1), (0,1,1,12)),
        ((1,0,0), (1,0,0,12))
    ]
    for order, sorder in sarima_orders:
        try:
            model = SARIMAX(
                ts,
                order=order,
                seasonal_order=sorder,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            fitted = model.fit(disp=False)
            return fitted
        except Exception:
            continue
    return None

# -----------------------------
# 3. Hybrid SARIMA + LSTM
# -----------------------------
hybrid_results = []
FORECAST_STEPS = 12  # forecast 12 months ahead

for cat in category_data['second-level_category'].unique():
    print(f"\nProcessing Category: {cat}")

    data_cat = category_data[category_data['second-level_category'] == cat]
    ts = data_cat.set_index('month')['sold/m']

    # Skip categories with insufficient variation
    if len(ts) < 18 or ts.sum() == 0 or ts.nunique() == 1:
        print(f"Skipping {cat}: not enough data or variation")
        continue

    # -----------------------------
    # SARIMA Model
    # -----------------------------
    sarima_fit = safe_sarima_fit(ts)
    if sarima_fit is None:
        print(f"⚠️ Skipping {cat}: SARIMA could not fit")
        continue

    sarima_forecast = sarima_fit.forecast(steps=FORECAST_STEPS)
    residuals = sarima_fit.resid

    # -----------------------------
    # LSTM on residuals
    # -----------------------------
    scaler = MinMaxScaler()
    scaled_res = scaler.fit_transform(residuals.values.reshape(-1,1))

    X, y = [], []
    for i in range(12, len(scaled_res)):
        X.append(scaled_res[i-12:i])
        y.append(scaled_res[i])
    X, y = np.array(X), np.array(y)

    lstm_model = Sequential([
        LSTM(32, activation='tanh', return_sequences=True, input_shape=(X.shape[1], 1)),
        LSTM(16, activation='tanh'),
        Dense(1)
    ])
    lstm_model.compile(optimizer='adam', loss='mse')

    es = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    lstm_model.fit(X, y, epochs=50, batch_size=8, verbose=0, callbacks=[es])

    # Predict future residuals
    last_sequence = scaled_res[-12:]
    residual_preds = []
    seq = last_sequence.copy()
    for _ in range(FORECAST_STEPS):
        inp = seq.reshape(1, 12, 1)
        pred = lstm_model.predict(inp, verbose=0)
        residual_preds.append(pred[0][0])
        seq = np.roll(seq, -1)
        seq[-1] = pred

    residual_forecast = scaler.inverse_transform(np.array(residual_preds).reshape(-1,1)).flatten()

    # -----------------------------
    # Hybrid forecast = SARIMA + LSTM residuals
    # -----------------------------
    hybrid_forecast = sarima_forecast.values + residual_forecast

    forecast_dates = pd.date_range(start=ts.index.max() + pd.offsets.MonthBegin(), 
                                   periods=FORECAST_STEPS, freq='MS')

    for d, f in zip(forecast_dates, hybrid_forecast):
        hybrid_results.append({
            'category': cat,
            'forecast_date': d,
            'predicted_sold_per_month': round(max(f,0),2)
        })

# -----------------------------
# 4. Save to CSV
# -----------------------------
hybrid_df = pd.DataFrame(hybrid_results)
hybrid_df.to_csv("hybrid_sarima_lstm_forecasts.csv", index=False)
print("\n✅ Hybrid forecasts saved to: hybrid_sarima_lstm_forecasts.csv")
hybrid_df.head()


Processing Category: Accessories


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)



Processing Category: Accessories Sets & Packages


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)



Processing Category: Additional Accessories


...

/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)



Processing Category: Writing & Correction


/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/georcelle/.pyenv/versions/3.12.5/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)



✅ Hybrid forecasts saved to: hybrid_sarima_lstm_forecasts.csv


,category,forecast_date,predicted_sold_per_month
0,Accessories,2025-12-01,38373677.98
1,Accessories,2026-01-01,37634495.54
2,Accessories,2026-02-01,37771404.22
3,Accessories,2026-03-01,38504379.21
4,Accessories,2026-04-01,38576794.80
